In [1]:
# load packages 
import numpy as np
import pandas as pd
import shutil

In [2]:
def create_temp(self, output_file, rSoil=0, hCritA = 1000000, rB = 0, \
                    hB=0, ht=0, tTop=0, tBot=0, Ampl=0, cBot=0):
    """
    Generates an ATMOSPH.IN file for HYDRUS 1D simulations, from a pandas dataframe.
    The column names of the dataframe must be:
    Precipitation - "Spilt_Irr+Precip.(cm)"
    Evaporation - "Evaporation(cm)"
    Transpiration - "Transpiration(cm)"
    Organic N - "flux_Organic.N.(g/cm3)"
    Ammonia N - "flux_Ammonia.N.(g/cm3)"
    Nitrate N - "flux_Nitrate.N.(g/cm3)"
    """
    num_days = len(self)

    hydrus_header = """Pcp_File_Version=4
        *** BLOCK I: ATMOSPHERIC INFORMATION  **********************************
        MaxAL                    (MaxAL = number of atmospheric data-records)
        {num_days}
        DailyVar  SinusVar  lLay  lBCCycles lInterc lDummy  lDummy  lDummy  lDummy  lDummy
        f       f       f       f       f       f       f       f       f       f
        hCritS                 (max. allowed pressure head at the soil surface)
        200
        tAtm Prec rSoil rRoot hCritA rB hB ht tTop tBot Ampl cTop cBot RootDepth"""

    hydrus_footer = """\nend*** END OF INPUT FILE 'ATMOSPH.IN' **********************************"""  

    with open(output_file,"w") as fout:
        fout.write(hydrus_header.format( num_days=num_days))

        for i in range(num_days):
            hydrus_row = ["\n", i+1, self["Spilt_Irr+Precip.(cm)"][i], self["Evaporation(cm)"][i], self["Transpiration(cm)"][i], hCritA, rB, hB, ht, tTop, tBot, Ampl, self["flux_Organic.N.(g/cm3)"][i], cBot, self["flux_Ammonia.N.(g/cm3)"][i], cBot, self["flux_Nitrate.N.(g/cm3)"][i], cBot]
            hydrus_row_text = " ".join([str(item) for item in hydrus_row])
            fout.write(hydrus_row_text)    

        fout.write(hydrus_footer)

In [3]:
def copyDirectory(src, dest):
    """
    copies the "src" file to the "dest"
    requires package shutil
    """
    
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)

In [4]:
def copy_setup(base_path, excel_file, orig_dir, copy_number):
    """
    Copys all files for a hydrus ID run, replaces the orginal ATMOSPH.IN file with new inputs from given excel file. 
    The orig_dir will be copied in the same base path location, with an associated executable file.
    The name of the copied will be the same as the orig_dir with the copy_number appended. 
    Expects an excel file that has sheets named "Sheet copy_number"
    If creating many new run files the excel_file should contain multipule sheets
    """
        
    # format original directory to append copy number
    to_append = orig_dir + "{}"
    
    # create new directory for run
    copyDirectory( base_path + orig_dir, base_path + to_append.format(copy_number))
    
    # create new initiation file with the same name
    shutil.copyfile(base_path + orig_dir + ".h1d" , base_path + to_append.format(copy_number) + ".h1d")
    
    # read in excel sheet
    sheet = "Sheet {}".format(copy_number)
    self = pd.read_excel(excel_file, sheet_name = sheet)

    # replace ATMOSPH.IN file
    out_file = base_path + to_append.format(copy_number) + "/ATMOSPH.IN"
    create_temp(self, output_file = out_file)

In [5]:
# for mac field
# set base path and excel file
base_path = "C:/Users/cmfmiller/Documents/Hydrus_Sims/MAC GW2/"
mac_excel = pd.ExcelFile("C:/Users/cmfmiller/Dropbox/Meyer Lab/Uncertainty Model/N Ratio/HYDRUS inputs/mac_app_reps.xlsx")

In [6]:
one = pd.read_excel(mac_excel, sheet_name = "Sheet 1")
one[1452:1460]

,Day.Number,Month,Date,Year,Acres,Evaporation(cm),Transpiration(cm),Precip.(cm),FW.(gal),PWW.(gal),...,Ammonia.N.(kg/cm2),split_Irrigation.(cm),split_Organic.N.(kg/cm2),split_Nitrate.N.(kg/cm2),split_Ammonia.N.(kg/cm2),Spilt_Irr+Precip.(cm),precip_minus_evap,flux_Ammonia.N.(g/cm3),flux_Nitrate.N.(g/cm3),flux_Organic.N.(g/cm3)
1452,1453,10,24,2015,67,0.052855,0.098185,0.00,1.134490e+07,671566.778371,...,7.465295e-07,3.272367,6.851709e-08,5.320549e-09,1.493059e-07,3.272367,3.219513,0.000046,0.000002,0.000021
1453,1454,10,25,2015,67,0.045062,0.088966,0.00,0.000000e+00,0.000000,...,0.000000e+00,3.272367,6.851709e-08,5.320549e-09,1.493059e-07,3.272367,3.227305,0.000046,0.000002,0.000021
1454,1455,10,26,2015,67,0.055302,0.115894,0.00,0.000000e+00,0.000000,...,0.000000e+00,3.272367,6.851709e-08,5.320549e-09,1.493059e-07,3.272367,3.217065,0.000046,0.000002,0.000021
1455,1456,10,27,2015,67,0.037102,0.082441,0.15,0.000000e+00,0.000000,...,0.000000e+00,3.272367,6.851709e-08,5.320549e-09,1.493059e-07,3.422367,3.385265,0.000044,0.000002,0.000020
1456,1457,10,28,2015,67,0.041169,0.096891,0.32,0.000000e+00,0.000000,...,0.000000e+00,3.272367,6.851709e-08,5.320549e-09,1.493059e-07,3.592367,3.551198,0.000042,0.000001,0.000019
1457,1458,10,29,2015,67,0.070623,0.175877,0.00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.070623,0.000000,0.000000,0.000000
1458,1459,10,30,2015,67,0.053105,0.139815,0.00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.053105,0.000000,0.000000,0.000000
1459,1460,10,31,2015,67,0.049382,0.137332,0.00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.049382,0.000000,0.000000,0.000000


In [7]:
# iterate over new runs
for it in range(1,1000+1):
    copy_setup(base_path, excel_file = mac_excel, orig_dir = "mac", copy_number = it)

In [5]:
# for men field
# set base path and excel file
men_path = "C:/Users/cmfmiller/Documents/Hydrus_Sims/MEN/"
men_excel = pd.ExcelFile("C:/Users/cmfmiller/Dropbox/Meyer Lab/Uncertainty Model/N Ratio/HYDRUS inputs/men_app_reps.xlsx")

In [6]:
men_path = "C:/Users/cmfmiller/Documents/Hydrus_Sims/MEN/"
one = pd.read_excel(men_excel, sheet_name = "Sheet 1")
one[1810:1820]

,Day.Number,Month,Date,Year,Acres,Evaporation(cm),Transpiration(cm),Precip.(cm),FW.(gal),PWW.(gal),...,Ammonia.N.(kg/cm2),split_Irrigation.(cm),split_Organic.N.(kg/cm2),split_Nitrate.N.(kg/cm2),split_Ammonia.N.(kg/cm2),Spilt_Irr+Precip.(cm),precip_minus_evap,flux_Ammonia.N.(g/cm3),flux_Nitrate.N.(g/cm3),flux_Organic.N.(g/cm3)
1810,1811,10,23,2016,40,0.117298,0.171209,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,-0.117298,0.0,0.0,0.0
1811,1812,10,24,2016,40,0.080914,0.116862,0.070,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.070,-0.010914,0.0,0.0,0.0
1812,1813,10,25,2016,40,0.076684,0.109585,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,-0.076684,0.0,0.0,0.0
1813,1814,10,26,2016,40,0.113049,0.159842,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,-0.113049,0.0,0.0,0.0
1814,1815,10,27,2016,40,0.021599,0.030215,0.040,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.040,0.018401,0.0,0.0,0.0
1815,1816,10,28,2016,40,0.039646,0.054867,1.345,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.345,1.305354,0.0,0.0,0.0
1816,1817,10,29,2016,40,0.094406,0.129250,1.355,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.355,1.260594,0.0,0.0,0.0
1817,1818,10,30,2016,40,0.068666,0.092996,1.825,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.825,1.756334,0.0,0.0,0.0
1818,1819,10,31,2016,40,0.080523,0.107872,1.345,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.345,1.264477,0.0,0.0,0.0
1819,1820,11,1,2016,40,0.103581,0.137252,0.070,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.070,-0.033581,0.0,0.0,0.0


In [7]:
# iterate over 1000 new runs
for it in range(1,1000+1):
    copy_setup(men_path, excel_file = men_excel, orig_dir = "men", copy_number = it)